<a href="https://colab.research.google.com/github/Crossme0809/whisper-tutorials/blob/main/PyDub_Spilit_Audio_for_Whisper_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **通过OpenAI Whipser API在线接口转录音频**


In [ ]:
!pip install openai --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.3 MB/s eta 0:00:00


In [ ]:
%pip install pydub

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "your-openai-api-key"

In [16]:
import openai
from pydub import AudioSegment
from pydub.silence import split_on_silence

openai.api_key = os.getenv("OPENAI_API_KEY")

# 将音频文件根据静音部分分割成块，并使用Whisper API进行语音识别的函数
def get_large_audio_transcription_on_silence_whisper(path, export_chunk_len, prompt):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    # 现在重新组合这些块，使得每个部分至少有export_chunk_len长。
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
        if len(output_chunks[-1]) < export_chunk_len:
            output_chunks[-1] += chunk
        else:
            # 如果最后一个输出块的长度超过目标长度， 我们可以开始一个新的块
            output_chunks.append(chunk)

    whole_text = ""
    for i, audio_chunk in enumerate(output_chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.mp3")
        audio_chunk.export(chunk_filename, format="mp3")

        try:
            audio_file = open(chunk_filename, "rb")
            transcript = openai.Audio.transcribe("whisper-1", audio_file, prompt=prompt)
            text = transcript['text']
        except Exception as e:
            print("Error:", str(e))
        else:
            text = f"{text.capitalize()}. "
            print(chunk_filename, ":", text)
            whole_text += text

    return whole_text

path="chatgpt_whisper.mp3"
export_chunk_len = 90 * 1000
prompt="这是一段中文视频，里面会介绍关于ChatGPT和Whisper实现语音转文字获取摘要的功能。"

audio_text = get_large_audio_transcription_on_silence_whisper(path, export_chunk_len, prompt)
print("\nAudio Full text:", audio_text)

audio-chunks/chunk1.mp3 : 大家好,在上次的影片中有稍微介绍到 如何去利用whisper将语音转换成文字 然后透过chatgpt去进行摘要 后来就有很多人问我讲说 那今天有没有办法去使用whisper的这个api 为影片来上字幕 也有人去问到讲说 诶,可是它在去使用我的这个程式的时候 里面好像有叫做ffmpeg没有办法去做安装 该怎么去做解决 以及有人去问到讲说 那whisper的api跟简映有怎样子的一个差别 好,所以这次我们在这一集的节目里面 我们去一一的去做个解答 好,所以首先来讲说呢 我们来看一下就是大家所关心的第一个问题就是 我今天有没有办法去使用whisper 来为影片来上字幕 那其实是可以的 当我们今天来讲说呢 点到这个就是whisper api的这个playground以后 其实是这个openai的这个就是api的playground以后 我们在这个documentation下面 我们可以寻找到audio的这个部分 那下面的时候呢 你可以看到讲说呢 在这里面我们可以去建立我们的这个就是转录档 那这里面讲说呢 它去吃几个参数 除了就是我们之前介绍的就是档案啦 模型之外 它另外讲说呢 有做response format 它这边呢 今天就可以从就是我们原本预设的这个json里面 我们可以改输出就是所谓的字幕档srt 甚至像是verbal json或者是vtt等这样子的一个档案 好,那我们就来看讲说呢 我们该怎么去实作这一段 所以呢 首先来讲说呢 我们去打开我们的jupyter notebook. 
audio-chunks/chunk2.mp3 : 不过上次来讲就有人提到讲说 他在自己的电脑上面去做这个程式码运行的时候 想要去把这个就是youtube影片下载下来以后去做一个转换 发现来讲说呢没有办法去做执行 主要呢原因是因为少了一个东西呢叫做ffmpeg 那ffmpeg呢它就是一个软体 你要把它给下载安装到你的自己电脑里面以后 我们今天才有办法去使用一些ytdlp的一些套件 来去进行这些youtube影片的这个下载跟做一些转换的一些动作 那如果说你没有去安装这个套件的话 那基本上来讲说呢上次的程式码会在这个地方就会产生卡关的问题 好那ffmpeg呢其实呢它的安装来讲说也不算复杂 你只要进到ffmpeg的这个官网以后呢你选择就是这个d

In [15]:
from concurrent.futures import ThreadPoolExecutor

# 定义要翻译的大文本
text = audio_text

# 将文本分割成较小的段落或句子
segments = []
segment_size = 800  # 每个段落的最大长度（以token为单位）

for i in range(0, len(text), segment_size):
    segment = text[i:i+segment_size]
    segments.append(segment)

# 使用线程池进行并发请求
executor = ThreadPoolExecutor(max_workers=5)  # 根据需要调整并发请求数量

# 上一个段落的翻译结果
previous_translation = ""
def translate_text_to_chinese(text):
    # 声明为全局变量
    global previous_translation

    # 添加上一个段落的翻译结果作为上下文信息
    translation = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"根据下面的上下文，将以下英文文本翻译成中文: Context: '{previous_translation}' '{text}'",
        max_tokens=1000,
    )

    translated_text = translation.choices[0].text.strip()
    previous_translation = translated_text

    return translated_text

# 提交并发请求（逐个翻译每个段落）
translated_segments = list(executor.map(translate_text_to_chinese, segments))

# 合并翻译结果
translated_text = ' '.join(translated_segments)

print(translated_text)

大家好，在上次的影片中我有介紹如何去利用Whisper將語音轉成文字，然後再利用Chaig GPT去進行摘要。之後有很多人問我如果有可能使用Whisper的API為影片製作字幕以及有關Whisper的API與簡映（快訊）的差別。 所以今天這集要一一去解答。首先，可以使用Whisper來為影片製作字幕，點擊whisper API的playground後，可以在documentation找到audio這個部分，這裡可以建立轉錄檔，他會吃參數，可以輸出字幕檔srt、verbal json或vtt等檔案。但是出現了一個問題，就是在安裝程式時缺少ffmpeg這個軟體。必須將ffmpeg下載安裝到電腦裡，之後才有辦法使用相關套件將YouTube AT上載影片轉換。 在這裡，我們對於下載YouTube影片以及對它進行一些轉換的動作，如果你沒有安裝ffmpeg這個套件的話，之前的程式碼會在此出現問題。ffmpeg的安裝雖然不是一鍵安裝，不過還是比較簡單的，你可以到官網去下載適用於不同作業系統版本的版本；安裝好以後，就可以使用ffmpeg進行操作了。如果今天覺得安裝ffmpeg過於複雜，可以考慮把程式碼放到codelab上，因為codelab預設ssh所開的vm都有預設裝載ffmpeg，所以我們可以在上面簡單地安裝ytdlp，下載YouTube的影片，並利用openai api對語音進行文字轉換。 翻译：我們將下載pip並安裝它，然後下載並安裝ytdlp、openai和pydub等相關套件。安裝完成後，我們將下載YouTube影片。我用codelab提供的表單稍加修改，就可以生成輸出檔案的名字。我們將Youtube連結輸入到右方選單裡。接著我們設定選項，將其輸出檔案名稱改為audio，並以mp3格式建立ytdop下載物件，最後再執行下載。最終在檔案序裡可以找到audio.mp3，然後使用quizper的api將音頻轉換成文字。由於quizper api的文件大小有限制，我們需要使用pydub來切割檔案，建議將檔案每個都切割為1000秒x1000解析度大約。 意思是100萬毫秒，因此我們今天將檔案分割成100萬毫秒，也就是1000秒。因此，如果這個影片超過20分鐘，它可能將分為16分鐘多一些的第一部分和約33分鐘的第二部分。為了做分割更方便，我們今天可以針對不同長度的影片做出不同的切割

In [ ]:
def translate_text_to_chinese(text):
    translation = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"将以下英文文本翻译成中文: '{text}'",
        max_tokens=1000,
    )
    translated_text = translation.choices[0].text.strip()

    return translated_text

In [10]:
chinese_audio_translation = translate_text_to_chinese(audio_text)
print("\nAudio Translate text:", chinese_audio_translation)


Audio Translate text: 你好，欢迎来到世界上最好的Python引导班。我叫安吉拉，是英国伦敦得分最高的编程引导培训学校 Appbury 的资深开发者和主要指导教师。迄今为止，我在现场和在线上教过超过50万名学生，我很高兴能作为你们的老师，教你们这门课程。作为本课程的学生，你们可以获得超过56小时的高清视频内容，其中包括一步一步的教程，互动编码练习，测验等等。 课程以100天的代码挑战为结构，所以你们可以期待着100天的坚持设计的内容，从Web开发到数据科学，涵盖了Python程序设计的方方面面。这是成为专业 Python 开发人员所需的唯一课程。每天上课时，你们都可以利用你们学到的东西来创建一个新项目。你们将构建一个机器人，在早晨给你发送短信，告诉你是否会下雨，这样你就永远不会忘记带雨伞了。你们将构建经典的街机游戏，像贪吃蛇和保龄球，以此来打动你的朋友，通过你自己设计的游戏来挑战他们。你们将学习理解复杂的数据，并创建出让你的同事印象深刻的美丽可视化图表。你们将创建一个程序，可以自动向你的朋友和家人发送生日祝福邮件.. 绝不会再忘记妈妈的生日了。你们将参与克隆现实世界初创公司的项目，廉价航班俱乐部？通过。建立你自己的博客？通过。推特机
